<body>
    <div>
        <h2 style='font-family: cursive; font-size: 40px;'>Método del Gradiente conjugado</h2>
<!--         <img src= alt=middle style="height: 340px"> -->
    </div>
</body>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Formulación matemática</h3>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Valores iniciales</h3>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Ventajas y desventajas</h3>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Pseudocódigo</h3>

<h3 style="font-family: 'Courier New'; font-size:25px">Octave</h3>

In [6]:
format long

function [g] = evaluar_gradiente(gradiente, varsym, vector)
    g = [];
    for i = 1:length(varsym)
        g = [g double(subs(gradiente(i), varsym, vector))];
    end
end

function [gradiente]=calcular_gradiente(footion,vars)
    %Esta funcion calcula el gradiente de la funcion de entrada.
    %
    %Sintaxis: calcular_gradiente(f,vars)
    %
    %Parámetros Iniciales:
    %            f = una cadena de caracteres (string) que representa a la función f.
    %            vars = Variables de la funcion.
    %
    %Parámetros de Salida: 
    %            gradiente = El gradiente de la funcion.
    %
    n=length(vars);    
    for i=1:n
        variables(i)=sym(vars(i));
    end
    for i=1:n
        gradiente(i)=diff(footion, variables(i));
    end
end


function [B_k] = seleccionaB(g, d, symvars, vectork, vector_k1, regla)
    if regla == 'FR'
        B_k = norm(double(subs(g, symvars, vector_k1)))**2/norm(double(subs(g, symvars, vectork)))**2;
    elseif regla == 'CD'
        B_k = norm(double(subs(g, symvars, vector_k1)))**2/(sum((-d) .* double(subs(g, symvars, vectork))));
    elseif regla == 'DY'
        yk = vector_k1 - vectork;
        B_k = norm(double(subs(g, symvars, vector_k1)))**2/(sum(d .* yk));
    end
end

function [vector_k1 e] = gradiente_conjugado(f, variables, vectork, tol, iter, regla)
    warning('off', 'all');
    pkg load symbolic
    
    footion = str2func(f);
    n = length(variables);
    symvars = [];
    e = [];
    
    for i=1:n
        symvars = [symvars sym(variables(i))];
    end
    g = calcular_gradiente(footion, symvars);
    g_num = evaluar_gradiente(g, symvars, vectork);
    d = -g_num;
    
    for k=1:iter
        alpha_k = 1;
        while double(subs(footion, symvars, vectork + alpha_k * d)) - double(subs(footion, symvars, vectork)) >= double(0.5*alpha_k*sum(g_num .* d))
            alpha_k = double(alpha_k / 2);
        end
        vector_k1 = vectork + alpha_k * d;
        error = double(norm(vector_k1 - vectork));
        e = [e error];
        if error < tol
            break;
        end
        g_num = evaluar_gradiente(g, symvars, vector_k1);
        B_k = double(seleccionaB(g, d, symvars, vectork, vector_k1, regla));
        d = double(-g_num + B_k*d);
        vectork = vector_k1;
    end
end

[res, e] = gradiente_conjugado('@(x,y,z)x ** 3 + y ** 3 + z ** 3 - 2 * x * y - 2 * x * z - 2 * y * z', ['x', 'y', 'z'], [1, 1, 1], 10^(-10), 10, 'FR')
     

Symbolic pkg v2.8.0: Python communication link active, SymPy v1.5.1.
g_num =

  -1  -1  -1

res =

   1.333331776323713   1.333331776323713   1.333331776323713

e =

 Columns 1 through 3:

   4.330127018922193e-01   8.880143300524028e-02   4.134592346876464e-02

 Columns 4 through 6:

   9.851792774233575e-03   3.093794738757982e-03   8.769852209933776e-04

 Columns 7 through 9:

   2.604984529655185e-04   7.567994980346868e-05   2.225586349537787e-05

 Column 10:

   6.507003381375837e-06



<h3 style="font-family: 'Courier New'; font-size:25px">Python</h3>

In [21]:
from sympy import symbols, sympify, diff, lambdify, Matrix
from sympy.matrices import matrix_multiply_elementwise

def calcular_funciones(f,vars):
    """
    Esta funcion calcula el gradiente de la funcion de entrada.
    Parametros iniciales:
    f = una cadena de caracteres (string) que representa a la función f.
    vars = variables de la funcion.
    
    Parámetros de Salida: 
    deetions = el gradiente de la funcion.
    """
    svars = symbols(vars)
    footion = sympify(f)
    deetions = []
    for var in svars:
        deetions.append(diff(footion, var))
    return footion, deetions, svars


def evaluar_gradiente(gradiente, varsym, vector):
    g = []
    vals = dict(zip(varsym, vector))
    for i in range(len(varsym)):
        g.append(gradiente[i].subs(vals))
    return Matrix(g)
    
    
def seleccionaB(g, gk, d, svars, vectork, vector_k1, regla):
    if regla == 'FR':
        B_k = gk.norm(2)**2/g.norm(2)**2
    elif regla == 'CD':
        B_k = gk.norm(2)**2/sum(-matrix_multiply_elementwise(d, g))
    elif regla == 'DY':
        yk = vector_k1 - vectork
        B_k = gk.norm(2)**2/(sum(matrix_multiply_elementwise(d, yk)))
    else:
        print('La regla ', regla, ' no es una regla valida')
    return B_k
    
    
def gradiente_conjugado(f, variables, vectork, tol, iterMax, regla):
    footion, gradiente, svars = calcular_funciones(f, variables)
    e = []
    g = evaluar_gradiente(gradiente, svars, vectork)
    d = -g
    error = tol + 1
    vectork = Matrix(vectork)
    
    for i in range(iterMax):
        alpha_k = 1
        while footion.subs(dict(zip(svars, vectork + alpha_k * d))) - footion.subs(dict(zip(svars, vectork))) >= 0.5 * alpha_k * sum(matrix_multiply_elementwise(g, d)):
            alpha_k /= 2
        
        vectork1 = vectork + alpha_k * d
        error = (vectork1 - vectork).norm(2)
        e.append(error)
        if error < tol:
            break
        gk = evaluar_gradiente(gradiente, svars, vectork1)
        B_k = seleccionaB(g, gk, d, svars, vectork, vectork1, regla)
        d = -gk + B_k * d
        vectork = vectork1
        g = gk
                                
    return vectork
    
print(gradiente_conjugado('(x - 2) ** 2 + (y + 3) ** 2 + x * y', ['x', 'y'], [1, 1], 10^(-10), 10, 'CD'))    
    
print(gradiente_conjugado('x ** 3 + y ** 3 + z ** 3 - 2 * x * y - 2 * x * z - 2 * y * z', ['x', 'y', 'z'], [1, 1, 1], 10^(-10), 10, 'DY'))
    
    

Matrix([[4.66647106708098], [-5.33313444327900]])
Matrix([[1.33333181270070], [1.33333181270070], [1.33333181270070]])
